Quick tool to get all different foods in the chinese document for pesticide residues. Used to create a list of selectable foods for the frontend of chipRAG.

**Info**: This code isn't needed for the ordinary chipRAG pipeline, but just a quick extraction tool for the (mockup) frontend. Also running it out of the /misc directory will lead to no success, as the imports are originally made from the root level -> so put it there before starting :)

In [ ]:
# PROMPT AND QUERY DEFINITION

## Query
query = """
    SELECT DISTINCT t.text
    FROM chinese_pesticide_residues as t
"""

## Prompt
prompt = """
You're about to get a section about a pesticide followed by a single or multiple tables with a bunch of foods possibly within a food category. These tables have the headings \"Food/Category Name\"
and \"Maximum Residue Limit\". Some of those Foods got no Maximum Residue Limit value next to them. Those are usually categories for foods or simply a food that goes over
multiple rows in the original document.
Your job is to extract all categories and foods from the section and put them into a python list like this: ["Category", "Food", "Category", "Food", "Food", "Category"].
There is no need to treat categories and foods different. Just put all of them into a list.
Add all you find in the section into the list. Don't add, alter or remove any values whatsoever.
Do never send/write anything back but the list.
Here is the section you are supposed to do that task with:
{text}
"""

In [ ]:
# IMPORT AND SETUP

import openai
from config.load_config import settings
from chiprag.postgres_utils import establish_connection, get_data

# setup openAI api
openai_client = openai.OpenAI(
        base_url=settings.kipitz_base_url,
        api_key=settings.kipitz_api_token
    )

# connect to database
conn, cur = establish_connection()

In [ ]:
# CLEAN SQL DATA AND GET VALUES

import json
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

raw_data = get_data(query)
data = [row[0].strip() for row in raw_data]

foods = []
save_every = 20
batch_size = 100 

def process_item(i, text):
    try:
        fprompt = prompt.format(text=text)
        completion = openai_client.chat.completions.create(
            model=settings.kipitz_model,
            messages=[{"role": settings.kipitz_role, "content": fprompt}],
        )
        parsed_list = json.loads(completion.choices[0].message.content)
        return i, parsed_list
    except Exception as e:
        print(f"Error at index {i}: {e}")
        return i, []

if not os.path.exists('tmp'):
    os.makedirs('tmp')

with ThreadPoolExecutor(max_workers=10) as executor: 
    future_to_index = {executor.submit(process_item, i, text): i for i, text in enumerate(data)}

    for idx, future in enumerate(as_completed(future_to_index)):
        i = future_to_index[future]
        try:
            i, result = future.result()
            foods.extend(result)
        except Exception as e:
            print(f"Exception at index {i}: {e}")

        if idx % save_every == 0:
            print(f"{idx}/{len(data)}")
            with open(f'tmp/tmpsave{idx}.txt', 'w') as f:
                for item in foods:
                    f.write(f"{item}\n")


with open('my_list.txt', 'w') as f:
    for item in foods:
        f.write(f"{item}\n")


In [ ]:
# REMOVING DUPLICATES, SAVING

with open('my_list.txt', 'r') as f:
    input = [line.strip() for line in f]


foods = [s.capitalize() for s in input]
foods = list(set(foods))
foods.sort()

with open('chinese_foods.txt', 'w') as f:
    for food in foods:
        f.write(f"{foods}\n")